In [1]:

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from math import sqrt

# Sample ratings data (user-item matrix)
data = {
    'User': ['U1', 'U1', 'U1', 'U2', 'U2', 'U3', 'U3', 'U3', 'U4', 'U4'],
    'Item': ['I1', 'I2', 'I3', 'I1', 'I4', 'I2', 'I3', 'I4', 'I2', 'I3'],
    'Rating': [5, 3, 4, 4, 2, 4, 5, 3, 2, 4]
}
df = pd.DataFrame(data)

# Create user-item matrix
ratings_matrix = df.pivot(index='User', columns='Item', values='Rating')
print("User-Item Matrix:\n", ratings_matrix)

# Fill missing ratings with 0 for similarity calculation
filled_matrix = ratings_matrix.fillna(0)

# Compute cosine similarity between users
similarity_matrix = pd.DataFrame(cosine_similarity(filled_matrix), 
                                 index=filled_matrix.index, 
                                 columns=filled_matrix.index)

print("\nUser Similarity Matrix:\n", similarity_matrix)

# Predict missing ratings for a specific user using weighted average
def predict_ratings(user_id, ratings_matrix, similarity_matrix):
    user_ratings = ratings_matrix.loc[user_id]
    unrated_items = user_ratings[user_ratings.isna()].index

    predicted_ratings = {}
    for item in unrated_items:
        sim_sum = 0
        weighted_sum = 0

        for other_user in ratings_matrix.index:
            if other_user != user_id and not pd.isna(ratings_matrix.loc[other_user, item]):
                sim = similarity_matrix.loc[user_id, other_user]
                weighted_sum += sim * ratings_matrix.loc[other_user, item]
                sim_sum += sim

        if sim_sum > 0:
            predicted_ratings[item] = weighted_sum / sim_sum
        else:
            predicted_ratings[item] = np.nan

    return predicted_ratings

# Recommend top items for User U4
user_id = 'U4'
predictions = predict_ratings(user_id, ratings_matrix, similarity_matrix)
predicted_series = pd.Series(predictions).sort_values(ascending=False)

print(f"\nTop Recommendations for {user_id}:\n{predicted_series}")

# Evaluation (optional RMSE calculation using known test set)
test_data = {
    'User': ['U4'],
    'Item': ['I1'],
    'ActualRating': [3]  # Suppose actual rating is known
}
test_df = pd.DataFrame(test_data)
test_df['PredictedRating'] = test_df.apply(
    lambda row: predict_ratings(row['User'], ratings_matrix, similarity_matrix).get(row['Item'], np.nan),
    axis=1
)

print("\nEvaluation on Test Data:")
print(test_df)

rmse = sqrt(mean_squared_error(test_df['ActualRating'], test_df['PredictedRating']))
print(f"\n RMSE on Test Data: {rmse:.4f}")


User-Item Matrix:
 Item   I1   I2   I3   I4
User                    
U1    5.0  3.0  4.0  NaN
U2    4.0  NaN  NaN  2.0
U3    NaN  4.0  5.0  3.0
U4    NaN  2.0  4.0  NaN

User Similarity Matrix:
 User        U1        U2        U3        U4
User                                        
U1    1.000000  0.632456  0.640000  0.695701
U2    0.632456  1.000000  0.189737  0.000000
U3    0.640000  0.189737  1.000000  0.885438
U4    0.695701  0.000000  0.885438  1.000000

Top Recommendations for U4:
I1    5.0
I4    3.0
dtype: float64

Evaluation on Test Data:
  User Item  ActualRating  PredictedRating
0   U4   I1             3              5.0

 RMSE on Test Data: 2.0000
